In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal

In [2]:
######################## define constants etc. ############################### 

workPath = '/n/holyscratch01/dvorkin_lab/Users/tianliwang/'

n_cosmo = 90

Om_d = 0.309
si_d = 0.816 

In [3]:
data = np.genfromtxt(workPath+'fullNet_testing_shuffle.dat', names=True, dtype=None)

In [4]:
Om_pred = data['Om_pred']
si_pred = data['si_pred']
Om_expect = data['Om_expect']
si_expect = data['si_expect']

# split the array into small arrays each corresponding the data from one cosmology (ex. array[0] is an array of all values in one cosmo)
Om_expect_split = np.array_split(Om_expect, n_cosmo)
Om_pred_split = np.array_split(Om_pred, n_cosmo)
si_expect_split = np.array_split(si_expect, n_cosmo)
si_pred_split = np.array_split(si_pred, n_cosmo)

In [35]:
np.cov([Om_pred_split[0], si_pred_split[0]])

array([[ 4.83841109e-05, -6.60729619e-05],
       [-6.60729619e-05,  5.84970176e-04]])